In [1]:
import csv
import pandas as pd
import gzip
import sys
import random as random
import math
import re
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [2]:
df_querys = pd.read_csv("msmarco-test2019-queries.tsv.gz", encoding="utf8",delimiter="\t",header=None)

In [3]:
df_querys.head()

,0,1
0,1108939,what slows down the flow of blood
1,1112389,"what is the county for grand rapids, mn"
2,792752,what is ruclip
3,1119729,what do you do when you have a nosebleed from ...
4,1105095,where is sugar lake lodge located


In [4]:
df_doctest_top100 = pd.read_csv("msmarco-doctest2019-top100.gz", encoding="utf8",delimiter="\t", header=None)

In [5]:
df_doctest_top100.head()

,0
0,1108939 Q0 D388799 1 -4.80563 IndriQueryLikeli...
1,1108939 Q0 D290852 2 -4.98713 IndriQueryLikeli...
2,1108939 Q0 D2449334 3 -4.9901 IndriQueryLikeli...
3,1108939 Q0 D884740 4 -5.03263 IndriQueryLikeli...
4,1108939 Q0 D2449337 5 -5.03325 IndriQueryLikel...


In [6]:
df_qrels = pd.read_csv("2019qrels.txt", encoding="utf8",delimiter="\t", header=None)

In [7]:
df_qrels.head()

,0
0,19335 Q0 D1035833 0
1,19335 Q0 D1068065 0
2,19335 Q0 D1068066 0
3,19335 Q0 D1088 0
4,19335 Q0 D109632 2


In [9]:
df_qrels.rename(columns={0: 'test'}, inplace= True)
df_qrels = pd.DataFrame( df_qrels.test.str.split(' ',3).tolist(),columns = ['query_id','i1','doc_id','i2'])

In [10]:
df_qrels.shape

(16258, 4)

In [11]:
a = df_qrels['query_id'].unique()
adict = {}

for queries in a:
    queryString = str(queries)
    b = df_qrels.loc[df_qrels['query_id'] == queryString]
    c = b['doc_id'].tolist() 
    
    adict[queries] = c
    

In [22]:
adict

AttributeError: 'dict' object has no attribute 'head'

In [13]:
df_doctest_top100.head()

,0
0,1108939 Q0 D388799 1 -4.80563 IndriQueryLikeli...
1,1108939 Q0 D290852 2 -4.98713 IndriQueryLikeli...
2,1108939 Q0 D2449334 3 -4.9901 IndriQueryLikeli...
3,1108939 Q0 D884740 4 -5.03263 IndriQueryLikeli...
4,1108939 Q0 D2449337 5 -5.03325 IndriQueryLikel...


In [14]:
df_doctest_top100.rename(columns={0: 'test'}, inplace= True)
df_doctest_top100 = pd.DataFrame( df_doctest_top100.test.str.split(' ',4).tolist(),columns = ['query_id','i1','doc_id','i2', 'i3'])

In [15]:
df_doctest_top100.head()

,query_id,i1,doc_id,i2,i3
0,1108939,Q0,D388799,1,-4.80563 IndriQueryLikelihood
1,1108939,Q0,D290852,2,-4.98713 IndriQueryLikelihood
2,1108939,Q0,D2449334,3,-4.9901 IndriQueryLikelihood
3,1108939,Q0,D884740,4,-5.03263 IndriQueryLikelihood
4,1108939,Q0,D2449337,5,-5.03325 IndriQueryLikelihood


In [16]:
x = df_doctest_top100['query_id'].unique()
bdict = {}

for queries in x:
    queryString = str(queries)
    y = df_qrels.loc[df_doctest_top100['query_id'] == queryString]
    z = y['doc_id'].tolist() 
    
    bdict[queries] = z

In [68]:
def compute_metrics(qids_to_relevant_documentids, qids_to_ranked_candidate_documents):
    """Compute MRR metric
    Args:    
    p_qids_to_relevant_documentids (dict): dictionary of query-document mapping
        Dict as read in with load_reference or load_reference_from_stream
    p_qids_to_ranked_candidate_documents (dict): dictionary of query-document candidates
    Returns:
        dict: dictionary of metrics {'MRR': <MRR Score>}
    """
    all_scores = {}
    MRR = 0
    qids_with_relevant_documents = 0
    ranking = []
    newList=[]
    cnt = 0
    
    for qid in qids_to_ranked_candidate_documents:
        if qid in qids_to_relevant_documentids:
            ranking.append(0)
            target_pid = qids_to_relevant_documentids[qid]
            candidate_pid = qids_to_ranked_candidate_documents[qid]
            for j in target_pid:
                j = str(j)
                j.strip().lower()
                newList.append(j)
            for i in range(0,len(candidate_pid)):
                n = str(candidate_pid[i])
                n.strip().lower()
                if n in newList:
                    MRR += 1/(i + 1)
                    print(1/(i+1))
                    ranking.pop()
                    ranking.append(i+1)
                    break
    if len(ranking) == 0:
        raise IOError("No matching QIDs found. Are you sure you are scoring the evaluation set?")
    
    MRR = MRR/len(qids_to_relevant_documentids)
    all_scores['MRR @100'] = MRR
    all_scores['QueriesRanked'] = len(set(qids_to_ranked_candidate_documents))
    return all_scores

In [69]:
w = compute_metrics(adict,bdict)

0.022222222222222223
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.017241379310344827
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.01098901098901099
1.0
1.0
1.0
1.0


In [70]:
w

{'MRR @100': 0.5593128514539902, 'QueriesRanked': 200}

In [21]:
print(len(adict))
print(len(bdict))

43
200


In [38]:
df_qrels.head()

,query_id,i1,doc_id,i2
0,19335,Q0,D1035833,0
1,19335,Q0,D1068065,0
2,19335,Q0,D1068066,0
3,19335,Q0,D1088,0
4,19335,Q0,D109632,2


## Testing removing all 0 qrels 

In [52]:
df_qrels_removed =  df_qrels[df_qrels.i2!= '0']

In [58]:
df_qrels_removed.head()

,query_id,i1,doc_id,i2
4,19335,Q0,D109632,2
5,19335,Q0,D109633,1
21,19335,Q0,D1254937,1
29,19335,Q0,D1347090,1
34,19335,Q0,D1372464,2


In [59]:
d = df_qrels_removed['query_id'].unique()
ddict = {}

for queries in d:
    queryString = str(queries)
    e = df_qrels_removed.loc[df_qrels_removed['query_id'] == queryString]
    f = e['doc_id'].tolist() 
    
    ddict[queries] = f

In [71]:
w = compute_metrics(ddict,bdict)

0.021739130434782608
0.25
1.0
1.0
0.3333333333333333
0.3333333333333333
0.1111111111111111
0.5
0.25
0.3333333333333333
0.2
0.25
1.0
0.5
0.01694915254237288
1.0
0.25
0.25
0.5
1.0
0.5
0.5
0.25
0.2
0.25
1.0


In [72]:
w

{'MRR @100': 0.27441393939740155, 'QueriesRanked': 200}

In [63]:
print(len(ddict))

43
